# Lab 2 - Creating and loading Bronze layer

## 2.1 - Testing connection with WWI_Bronze

In [ ]:
-- step 2.1.1
SELECT DB_NAME() AS DatabaseName

## 2.2 - Create bronze layer tables

In [ ]:
--step 2.2.1
DROP TABLE IF EXISTS dbo.Customers;
DROP TABLE IF EXISTS dbo.StockItems;
DROP TABLE IF EXISTS dbo.Invoices;
DROP TABLE IF EXISTS dbo.InvoiceLines;


In [ ]:
--step 2.2.2
CREATE TABLE [dbo].[Customers]
(
    [CustomerID]          [int]           NULL,
    [CustomerName]        [varchar](100)  NULL,
    [ContactPerson]       [varchar](50)   NULL,
    [PhoneNumber]         [varchar](20)   NULL,
    [WebsiteURL]          [varchar](256)  NULL,
    [DeliveryAddressLine] [varchar](60)   NULL,
    [DeliveryPostalCode]  [varchar](10)   NULL,
    [LastEditedWhen]      [datetime2](6)  NULL
);
GO

CREATE TABLE [dbo].[InvoiceLines]
(
    [InvoiceID]      [int]            NULL,
    [InvoiceLineID]  [int]            NULL,
    [StockItemID]    [int]            NULL,
    [Quantity]       [int]            NULL,
    [UnitPrice]      [decimal](18, 2) NULL,
    [TaxRate]        [decimal](18, 3) NULL,
    [TaxAmount]      [decimal](18, 2) NULL,
    [LineProfit]     [decimal](18, 2) NULL,
    [LastEditedWhen] [datetime2](6)   NULL
);
GO

CREATE TABLE [dbo].[Invoices]
(
    [InvoiceID]      [int]          NULL,
    [InvoiceDate]    [datetime2](6) NULL,
    [CustomerID]     [int]          NULL,
    [LastEditedWhen] [datetime2](6) NULL
);
GO

CREATE TABLE [dbo].[StockItems]
(
    [StockItemID]     [int]            NULL,
    [StockItemName]   [varchar](100)   NULL,
    [Brand]           [varchar](50)    NULL,
    [PackageTypeName] [varchar](50)    NULL,
    [TaxRate]         [decimal](18, 3) NULL,
    [UnitPrice]       [decimal](18, 2) NULL,
    [Tags]            [varchar](max)   NULL,
    [LastEditedWhen]  [datetime2](6)   NULL
);
GO


In [ ]:
--step 2.2.3
SELECT
    SCHEMA_NAME(schema_id) AS SchemaName,
    name AS TableName
FROM sys.tables
WHERE
    SCHEMA_NAME(schema_id) IN ('dbo')
ORDER BY
    SchemaName,
    TableName

## Step 2.3 - Create stored procedures for loading data into bronze layer

In [ ]:
--step 2.3.1
CREATE OR ALTER PROCEDURE dbo.sp_loadCustomers
AS
BEGIN
    TRUNCATE TABLE dbo.Customers;
    COPY INTO dbo.Customers
    FROM 'https://fabricwhtutorialdata.blob.core.windows.net/sampledata/WideWorldImporters/csv/Customers/*.csv'
    WITH (
        FILE_TYPE = 'CSV',
        FIRSTROW = 2
    );
END
GO

CREATE OR ALTER PROCEDURE dbo.sp_loadStockItems
AS
BEGIN
    TRUNCATE TABLE [dbo].[StockItems];
    COPY INTO dbo.StockItems
    FROM 'https://fabricwhtutorialdata.blob.core.windows.net/sampledata/WideWorldImporters/parquet/StockItems/*.parquet'
    WITH (
        FILE_TYPE = 'PARQUET'
    );
END
GO

CREATE OR ALTER PROCEDURE dbo.sp_loadInvoices
AS
BEGIN
    TRUNCATE TABLE [dbo].[Invoices];
    INSERT INTO dbo.Invoices
    (
        [InvoiceID],
        [InvoiceDate],
        [CustomerID],
        [LastEditedWhen]
    )
    SELECT
        [InvoiceID],
        [InvoiceDate],
        [CustomerID],
        [LastEditedWhen]
    FROM OPENROWSET(BULK 'https://fabricwhtutorialdata.blob.core.windows.net/sampledata/WideWorldImporters/parquet/Invoices/*.parquet');
END
GO

CREATE OR ALTER PROCEDURE dbo.sp_loadInvoiceLines
AS
BEGIN
    TRUNCATE TABLE [dbo].[InvoiceLines];
    COPY INTO dbo.InvoiceLines
    FROM 'https://fabricwhtutorialdata.blob.core.windows.net/sampledata/WideWorldImporters/parquet/InvoiceLines/*.parquet'
    WITH (
        FILE_TYPE = 'PARQUET'
    );
END
GO

In [ ]:
--step 2.3.2
SELECT
    SCHEMA_NAME(p.schema_id) AS SchemaName,
    p.name AS ProcedureName
FROM
    sys.procedures AS p
WHERE
    SCHEMA_NAME(p.schema_id) = 'dbo'
ORDER BY
    SchemaName, ProcedureName

### Step 2.4 - Execute procedures to load the data

In [ ]:
--step 2.4.1
DECLARE @CountBeforeLoadCustomers  BIGINT = (SELECT COUNT_BIG(*) FROM dbo.Customers)
DECLARE @CountBeforeLoadStockItems     BIGINT = (SELECT COUNT_BIG(*) FROM dbo.StockItems)
DECLARE @CountBeforeLoadInvoices BIGINT = (SELECT COUNT_BIG(*) FROM dbo.Invoices)
DECLARE @CountBeforeLoadInvoiceLines     BIGINT = (SELECT COUNT_BIG(*) FROM dbo.InvoiceLines)
   
EXEC dbo.sp_loadCustomers;
EXEC dbo.sp_loadStockItems;
EXEC dbo.sp_loadInvoices;
EXEC dbo.sp_loadInvoiceLines;

SELECT 'dbo'   AS SchemaName, 'Customers'       AS TableName, FORMAT(@CountBeforeLoadCustomers,     'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.Customers        UNION ALL
SELECT 'dbo'   AS SchemaName, 'StockItems'    AS TableName, FORMAT(@CountBeforeLoadStockItems,  'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.StockItems     UNION ALL
SELECT 'dbo'   AS SchemaName, 'Invoices'   AS TableName, FORMAT(@CountBeforeLoadInvoices, 'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.Invoices    UNION ALL
SELECT 'dbo'   AS SchemaName, 'InvoiceLines'       AS TableName, FORMAT(@CountBeforeLoadInvoiceLines,     'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.InvoiceLines
ORDER BY
    SchemaName,
    TableName